In [22]:
#!/home/maghoi/.conda/envs/py37/bin/python
from Bio import SeqIO
import argparse, os, glob

#parser = argparse.ArgumentParser(description="Removes duplicate seqs from input file, writes to input_out")
#parser.add_argument("-i", dest="INPUT_FASTA", required=True, help="Input fasta file")
#parser.add_argument("-o", dest="OUTPUT_FASTA", help="Output fasta file")
#args = parser.parse_args()  
#dict_args = vars(args)


def de_duplicate_FASTA_files(inDir="../data/raw/", v=1):
    # Find train and test fasta files in raw
    fasta_files = glob.glob(inDir + "test*.fasta") + glob.glob(inDir + "train*.fasta")
    print("Checking for duplicate sequences in:\n", fasta_files, end="\n")
    outDir = "../data/interim/"

    # Count duplicates and entries across all fasta files
    dict_seqs = {}
    dict_ids = {}
    count_entries = 0

    # Load individual FASTA files and check whether sequences are duplicates before saving
    # IDs are only stored for counting
    for inFasta in fasta_files:
        outFasta = outDir + "filtered_" + os.path.basename(inFasta)

        with open(outFasta, 'w') as outHandle:
            no_duplicates = 0
            for record in SeqIO.parse(inFasta, "fasta"):
                count_entries += 1

                # Store unique sequences for later checking
                # Write ID+sequence if sequence is unique
                if str(record.seq) not in dict_seqs:
                    dict_seqs[str(record.seq)] = True
                    SeqIO.write(record, outHandle, 'fasta')

                    # Store unique IDs
                    if str(record.id) not in dict_ids:
                        dict_ids[str(record.id)] = True

                else:
                    no_duplicates += 1

            if v:
                print(f"Found {no_duplicates} duplicate sequences in {inFasta}")
                print(f"Saving to {outFasta}")
                
    # Print statistics
    print(f"\nProcessed {count_entries} sequences from {len(fasta_files)} train/test FASTA files in {inDir} to {outDir}:")
    print(f"Unique IDs / Sequences: {len(dict_ids.keys())} / {len(dict_seqs.keys())}")

de_duplicate_FASTA_files()

Checking for duplicate sequences in:
 ['../data/raw/test_neg.fasta', '../data/raw/test_pos.fasta', '../data/raw/train_pos.fasta', '../data/raw/train_neg.fasta']
Found 0 duplicate sequences in ../data/raw/test_neg.fasta
Saving to ../data/interim/filtered_test_neg.fasta
Found 0 duplicate sequences in ../data/raw/test_pos.fasta
Saving to ../data/interim/filtered_test_pos.fasta
Found 13 duplicate sequences in ../data/raw/train_pos.fasta
Saving to ../data/interim/filtered_train_pos.fasta
Found 256 duplicate sequences in ../data/raw/train_neg.fasta
Saving to ../data/interim/filtered_train_neg.fasta

Processed 3246 sequences from 4 train/test FASTA files in ../data/raw/ to ../data/interim/:
Unique IDs / Sequences: 2977 / 2977


In [24]:
call("" + " myarg", shell=True)

'nonamp5_30_1529'

In [ ]:
call("${PYTHON_INTERPRETER} -u " + "src/data/extract.py " + "", shell=True)

In [46]:
import subprocess
subprocess.run("echo hello world", shell=True, capture_output=True)

CompletedProcess(args='echo hello world', returncode=0, stdout=b'hello world\n', stderr=b'')

In [47]:
for fastaFile in glob.glob("../data/interim/test.fasta"):
    print(fastaFile)

    fastaName = Path(os.path.basename(fastaFile)).stem
    script = "${PYTHON_INTERPRETER} -u extract.py"
    model= "esm1_t6_43M_UR50S"
    params = "include per_tok"
    outDir = "../data/interim/" + fastaName + "/"

    cmd = " ".join([script, model, fastaFile, outDir, params])
    print(cmd)
    subprocess.run(cmd, shell=True, capture_output=True)

../data/interim/test.fasta
${PYTHON_INTERPRETER} -u extract.py esm1_t6_43M_UR50S ../data/interim/test.fasta ../data/interim/test/ include per_tok


In [53]:
import subprocess
with open('../src/data/run_esm.sh', 'rb') as file:
    script = file.read()
subprocess.run("bash ../src/data/run_esm.sh", shell=True)

CompletedProcess(args='bash ../src/data/run_esm.sh', returncode=2)

In [55]:
!pwd

/Users/maghoi/repos/__personal/MLOps_sequences/notebooks


In [58]:
!realpath bash ../src/data/run_esm.sh

/Users/maghoi/repos/__personal/MLOps_sequences/notebooks/bash
/Users/maghoi/repos/__personal/MLOps_sequences/src/data/run_esm.sh


In [49]:
import os, glob
from collections import OrderedDict

import numpy as np
import torch

from Bio import SeqIO

def stack_res_tensors(tensors):
    embedding_dim = tensors[0].shape[-1]

    # Prepare to stack tensors
    n_seqs = len(tensors)
    seq_max_length = max([t.shape[0] for t in tensors])

    # Initialize empty padded vector, with 0-padding for sequences with less than max length residues
    fill_tensor = torch.zeros(size=(n_seqs, seq_max_length, embedding_dim))
    
    # Load torch tensors from ESM embeddings matching sequence, fill padded tensor
    for i, tensor in enumerate(tensors):
        fill_tensor[i, 0:tensor.shape[0]] = tensor

    return(fill_tensor)

def PCA_transform_stacked_tensor(stacked_tensor, pca=None, pca_dim=30, v=True):
    
    """
    Fits PCA and transforms on flattened dataset representing all positions x ESM_embeddings, before reshaping back to n_seqs x positions x pca_dim

    Returns:
    (pca_stacked_tensor) [torch.Tensor]: Dataset reduced to n_seqs x sequence_length x pca_dim (30)
    """
    embedding_dim = stacked_tensor.shape[-1]

    # Flatten training dataset to shape n_positions/residues x embedding_dim
    positions = stacked_tensor.view( stacked_tensor.shape[0]*stacked_tensor.shape[1], embedding_dim )

    # PCA reduce to n_dim (20) dimensions along positions
    if pca is None:
        print("Fitting and transforming PCA")
        from sklearn.decomposition import PCA
        pca = PCA(pca_dim)
        positions_reduced = pca.fit_transform(positions)
    else:
        print("Using given pca object to transform")
        positions_reduced = pca.transform(positions)

    # Reshape dataset to n_seqs x n_positions x 30
    pca_stacked_tensor = positions_reduced.reshape( stacked_tensor.shape[0], stacked_tensor.shape[1], pca_dim )
    pca_stacked_tensor = torch.Tensor(pca_stacked_tensor)

    if v:
        print(f"Reduced from {stacked_tensor.shape} to {pca_stacked_tensor.shape}")

    return(pca_stacked_tensor, pca)

def save_processed_tensor(tensor_proc, id_tensor_odict, outdir, verbose=0):
    
        # Create new dict with original shape of PCA reduced tensors (remove zero-øadding in processed stacked tensor)
        pca_dict = {}
        for i, t_id in enumerate(id_tensor_odict.keys()):
                orig_tensor = id_tensor_odict[t_id]
                pca_tensor = tensor_proc[i]
                
                # Fill with PCA reduced tensor, preserving original shape
                pca_dict[t_id] = pca_tensor[0:orig_tensor.shape[0], 0:orig_tensor.shape[1]]

        # Save
        print("Writing PCA-reduced tensors to %s" % outdir)
        os.makedirs(outdir, exist_ok=True)
        for t_id, t in pca_dict.items():
                outpath = outdir + str(t_id) + ".pt"

                if verbose: print("Writing ID %s %s to %s" % (t_id, t.shape, outpath))

                # NB: tensor.clone() ensures tensor is stored as its actual slice and not as 37 mb tensor represented in memory
                # Difference of e.g. 20 kb vs 37 mb
                torch.save(t.clone(), outpath)

        return pca_dict

In [50]:
inDir = "../data/raw/"
train_pos_list = glob.glob(inDir + "train/pos/*.fasta")
train_neg_list = glob.glob(inDir + "train/neg/*.fasta")
test_pos_list = glob.glob(inDir + "test/pos/*.fasta")
test_neg_list = glob.glob(inDir + "test/neg/*.fasta")


In [59]:
stacked_tensor.shape

torch.Size([1, 25, 768])

In [180]:
procDir = "../data/processed/"
pca_dim=60

In [207]:
def get_PCA_fit_train():
    """ Returns fitted PCA object on training dataset """

    def PCA_fit(stacked_tensor, pca_dim=60):
        """ Returns fitted PCA on flattened positions (N x L x D) -> PCA(N x LD) """

        # Flatten training dataset to shape n_positions/residues x embedding_dim
        embedding_dim = stacked_tensor.shape[-1]
        positions = stacked_tensor.view( stacked_tensor.shape[0]*stacked_tensor.shape[1], embedding_dim )

        from sklearn.decomposition import PCA
        pca = PCA(pca_dim)
        
        return(pca.fit(positions))
        
    tensor_files = glob.glob(interimDir + "train_pos/*.pt") + glob.glob(interimDir + "train_neg/*.pt")

    for file in tensor_files:
        tensor_dict = torch.load(file)
        id_tensor_odict[tensor_dict["label"]] = tensor_dict["representations"][6]

    # PCA fit
    tensors = [tensor for tensor in id_tensor_odict.values()]
    stacked_tensor = stack_res_tensors(tensors)

    embed_dim = stacked_tensor.shape[-1]
    max_len = len(stacked_tensor[0])

    print(f"Fitting PCA on train dataset {stacked_tensor.shape}")
    pca = PCA_fit(stacked_tensor, pca_dim)

    return pca, embed_dim, max_len


In [208]:
pca_train, embed_dim, max_len = get_PCA_fit_train()

In [211]:
dirType_list = ["test_pos/", "test_neg/", "train_pos/", "train_neg/"]
fasta_list = ["test_pos.fasta", "test_neg.fasta", "train_pos.fasta", "train_neg.fasta"]

def PCA_transformed_save_embed_files(dirType_list, fasta_list, interimDir, procDir):
    print(f"PCA transforming and saving embedding pt files in {dirType_list}")
    # PCA transform
    id_tensor_odict = OrderedDict()

    for dirType in dirType_list:
        os.makedirs(procDir + dirType, exist_ok=True)

    for fasta, dirType in zip(fasta_list, dirType_list):

        # Load FASTA file for ids
        record_dict = SeqIO.to_dict(SeqIO.parse(interimDir + fasta, "fasta"))

        # Load, transform and tensor files
        tensor_files = glob.glob(interimDir + dirType + "*.pt")

        for file in tensor_files:
            name = os.path.basename(file)

            t_dict = torch.load(file)
            id = t_dict["label"]

            # PCA transform tensor
            tensor = t_dict["representations"][6]
            
            arr = pca.transform(tensor)
            padded_arr = np.zeros( (max_len, pca_dim) )
            padded_arr[ :len(arr) ] = arr
            pca_tensor = torch.from_numpy(padded_arr).type(torch.float32)

            value_dict = {
                        "tensor": pca_tensor,
                        "sequence": str(record_dict[id].seq),
                        "length": len(arr)
                        }

            outFile = procDir + dirType + name
            torch.save(value_dict, outFile)

In [212]:
PCA_transformed_save_embed_files(dirType_list, fasta_list, interimDir, procDir)

PCA transforming and saving embedding pt files in ['test_pos/', 'test_neg/', 'train_pos/', 'train_neg/']


In [144]:
t_dict["representations"][6].dtype

torch.float32

In [150]:
arr = pca.transform(tensor)
pca_tensor = torch.from_numpy(arr).type(torch.float32)

In [96]:
# Load in all FASTA ids
fasta, dir, label = "test_pos.fasta", "test_pos/", 1
id_odict = OrderedDict()

fasta_list = ["test_pos.fasta", "test_neg.fasta", "train_pos.fasta", "train_neg.fasta"]
dir_list = ["test_pos/", "test_neg/", "train_pos/", "train_neg/"]
labels = [1, 0, 1, 0]

for fasta, dir, label in zip(fasta_list, dir_list, labels):
    for record in SeqIO.parse(interimDir + fasta, "fasta"):
        id_odict[record.id] = {
                                "sequence": str(record.seq),
                                "label": label
                              }

In [97]:
interimDir = "../data/interim/"

# Training set
print("PCA reducing train set by fitting and transforming")
train_pos_list = glob.glob(interimDir + "test_pos/" + "*.pt")
train_neg_list = glob.glob(interimDir + "test_neg/" + "*.pt")

id_tensor_odict = OrderedDict()
for train_list in [train_pos_list, train_neg_list]:
    for file in train_list:
        t_dict = torch.load(file)
        id = t_dict["label"]

        value_dict = {
                    "tensor": t_dict["representations"][6],
                    "sequence": id_odict[id]["sequence"],
                    "label": id_odict[id]["label"]
                     }

        id_tensor_odict[id] = value_dict

# PCA fit and transform along positions
tensors = [value_dict["tensor"] for value_dict in id_tensor_odict.values()]
stacked_tensor = stack_res_tensors(tensors)
pca_stacked_tensor, pca_train = PCA_transform_stacked_tensor(stacked_tensor, pca=None, pca_dim=60)

# Save


# Test set
print("PCA reducing test set using fitted train PCA")
test_pos_list = glob.glob(interimDir + "test_pos/" + "*.pt")
test_neg_list = glob.glob(interimDir + "test_neg/" + "*.pt")

id_tensor_odict = OrderedDict()
for train_list in [train_pos_list, train_neg_list]:
    for file in train_list:
        tensor_dict = torch.load(file)
        id_tensor_odict[tensor_dict["label"]] = tensor_dict["representations"][6]

# PCA Transform using pca_train along positions
tensors = [tensor for tensor in id_tensor_odict.values()]
stacked_tensor = stack_res_tensors(tensors)
pca_stacked_tensor, pca_train = PCA_transform_stacked_tensor(stacked_tensor, pca=pca_train, pca_dim=60)

PCA reducing train set by fitting and transforming
Fitting and transforming PCA
Reduced from torch.Size([188, 30, 768]) to torch.Size([188, 30, 60])
PCA reducing test set using fitted train PCA
Using given pca object to transform
Reduced from torch.Size([188, 30, 768]) to torch.Size([188, 30, 60])


IndexError: list index out of range

In [ ]:
pca_train

PCA(n_components=60)

In [ ]:
a = stacked_tensor.shape
b = torch.transpose(stacked_tensor, )
print(a, b)

torch.Size([188, 30, 768])

In [ ]:
stacked_tensor.shape

torch.Size([188, 30, 768])

In [ ]:
id_tensor_odict

OrderedDict([('unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_ampepTr_cdh8_sample94_88',
              tensor([[-0.9734,  0.4780, -1.2807,  ..., -1.2562,  0.6164,  0.1830],
                      [-0.7679, -1.1575, -1.2707,  ..., -0.5380,  1.3922, -0.6548],
                      [-0.3066,  0.2908, -0.8684,  ..., -0.0237,  0.1824, -0.4035],
                      ...,
                      [-0.3776,  0.1662, -0.1659,  ..., -0.0407,  1.1896, -0.3668],
                      [-0.3426, -0.0325,  0.2227,  ...,  0.5822,  0.1860, -0.4921],
                      [-0.5355, -0.1149,  0.3087,  ..., -0.1021,  1.1296, -0.4966]])),
             ('unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_ampepTr_cdh8_sample94_1',
              tensor([[-0.5702, -0.5702,  0.0291,  ...,  0.0808,  0.8041,  1.1573],
                      [-1.3881, -1.1725,  1.5506,  ...,  1.0189,  1.3763,  0.6525],
                      [-1.0457, -0.3658,  0.4372,  ...,  0.1318,  0.7816,  0.3555],
                      ...,
            

In [ ]:
file = tensor_files[0]
torch.load(file)["representations"][6]

tensor([[-0.9734,  0.4780, -1.2807,  ..., -1.2562,  0.6164,  0.1830],
        [-0.7679, -1.1575, -1.2707,  ..., -0.5380,  1.3922, -0.6548],
        [-0.3066,  0.2908, -0.8684,  ..., -0.0237,  0.1824, -0.4035],
        ...,
        [-0.3776,  0.1662, -0.1659,  ..., -0.0407,  1.1896, -0.3668],
        [-0.3426, -0.0325,  0.2227,  ...,  0.5822,  0.1860, -0.4921],
        [-0.5355, -0.1149,  0.3087,  ..., -0.1021,  1.1296, -0.4966]])

In [ ]:
os.path.basename(file)

'unip30_cdh10_stdif_dpAmpepTr30_iamp2l_ampScan_ampepTr_cdh8_sample94_88.pt'

In [ ]:
# Read in ESM tensor files to ordered dict
id_tensor_odict = OrderedDict()
tensor_files = glob.glob(args.EMBEDDING_DIR + "filtered_train*/*.pt")

if args.VERBOSE: print("Found %s ESM tensor files in %s" % (len(tensor_files), args.EMBEDDING_DIR))

for file in tensor_files:
    esm_d = torch.load(file)
    id_tensor_odict[esm_d["label"]] = esm_d["representations"][33]

# Stack with zero-padding by longest sequence length
tensors = [tensor for tensor in id_tensor_odict.values()]
stacked_tensor = stack_res_tensors(tensors)
if args.VERBOSE: print("Stacked tensor:", stacked_tensor.shape)

# PCA reduce to 30 dimensions
if args.VERBOSE: print("Fitting PCA model to stacked_tensor ...")
pca_stacked_tensor = PCA_reduce_stacked_tensor(stacked_tensor, pca_dim=30)
if args.VERBOSE: print("PCA stacked tensor:", pca_stacked_tensor.shape)


# Save to new files
pca_dict = save_processed_tensor(pca_stacked_tensor, id_tensor_odict, outdir=args.OUTDIR, verbose=args.VERBOSE)